In [1]:
import dropbox
import os
import numpy as np
import pandas as pd
from scipy.stats import t
import math
import collections
from PIL import Image
from dropbox import DropboxOAuth2FlowNoRedirect
import matplotlib.pyplot as plt
import cv2
import pypyodbc
from sklearn.preprocessing import LabelEncoder
import sys
import traceback

from keras.applications.xception import Xception
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model, Sequential
from keras.layers import BatchNormalization, concatenate, Input, Dropout, Flatten, Dense
from keras.utils.np_utils import to_categorical  #?

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
cnxn = pypyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                      "Server=028-GUMUS1-SRV;"
                      "Database=KOTON_DB;"
                      "Trusted_Connection=yes;")

In [3]:
access_token='0DQLcsXIojcAAAAAAAgybeB3DOmXKK7bRTLyYwkthbrAiGmpQR4AuGINQkjBXhif'
dbx = dropbox.Dropbox(access_token)

In [4]:
retrieved_image_df = pd.read_excel('C:/Users/Recep/Koton/Data/retrieved_image_list.xlsx', sheet_name='Sheet1')

In [5]:
def data_set(codes, indexes):
    """Takes list of option codes as input and returns related dataset, response set (sales amount)
    and genuine option codes that included in e-commerce table.
    """
    
    if len(codes)>2000:
        numOfBatches = 100
    else:
        numOfBatches = 1
    
    string_tar = "Select OptionCode, TTSales = ISNULL(sum(TotalSales), 0) \
    from (Select CODE, TotalSales \
    From ( \
    SELECT [CODE], TotalSales=Sum(isnull(cast(totalqty as float),0)) \
      FROM [KOTON_DB].[dbo].[FA_Sales_Ecommerce] Group by CODE)t) AS Sales INNER JOIN [KOTON_DB].[dbo].[LU_Variant_New] AS Variant ON Variant.SkuCode = Sales.Code \
    Where "
    
    codes_split = np.array_split(codes, numOfBatches)
    indexes_split = np.array_split(indexes, numOfBatches)
    
    # to prevent empty targets
    gen_codes = []
    gen_indexes = []
    target = []
    
    for batch in range(numOfBatches):
        _ =[]
        flag2 = True
        for i in codes_split[batch]:
            if flag2:
                _.append("OptionCode='"+i+"'")
                flag2 = False
            else:
                 _.append(" OR OptionCode='"+i+"'")
        query2 = string_tar + ''.join(_) + " Group by OptionCode"
        temp = pd.read_sql_query(query2, cnxn, index_col='optioncode')
        target.append(temp)
    target = pd.concat(target)

    for code, index in zip(codes, indexes):
        try:
            target.loc[code]
            gen_codes.append(code)
            gen_indexes.append(index)
        except:
            pass        
        
    
    string_inp = "SELECT DISTINCT OptionCode, ProductHierarchyLevel02, ProductHierarchyLevel03, ProductHierarchyLevel04, ProductHierarchyLevel05, SeasonCode, KOLEKSİYON, [KULLANIM-eski], [MODEL TASARIMI], [OZEL KOLEKSIYON], SEZONU \
  FROM [KOTON_DB].[dbo].[LU_Model_New] AS Model \
  INNER JOIN [KOTON_DB].[dbo].[LU_Variant_New] AS Variant ON Variant.ItemCode = Model.ItemCode \
  WHERE "
    
#     numOfBatches = max(min(len(gen_indexes), numOfBatches), 1)
    numOfBatches = max(min(len(gen_indexes), numOfBatches), 1)
    gen_codes_splited = np.array_split(gen_codes, numOfBatches)
    dataset = []
    
    for batch in range(numOfBatches):
        _ =[]
        flag1 = True
        for i in gen_codes_splited[batch]:
            if flag1:
                _.append("OptionCode='"+i+"'")
                flag1 = False
            else:
                 _.append(" OR OptionCode='"+i+"'")
        query1 = string_inp + ''.join(_)
        dataset.append(pd.read_sql_query(query1, cnxn, index_col='optioncode'))
    
    dataset = pd.concat(dataset)
    
    
    return dataset, target, gen_codes, gen_indexes

In [6]:
number_of_samples = 50
startIndex = 16000

In [85]:
def getImages_n_train(numberSamples, startIndex, batchSize=64):
    errorLog = []
    bn_model = Xception(include_top=False, weights='imagenet')  #, pooling='avg'
    
    first_input = Input(shape=(10, 10, 2048))
    first = Flatten()(first_input)
    
    second = Input(shape=(10,)) #change according to columns
#     second = Dense(10, activation='relu')(second_)
    
    model = concatenate([first, second], axis=1)
    model = BatchNormalization()(model)
    model = Dense(256, activation='relu')(model)
    model = Dropout(0.5)(model)
    model = Dense(256, activation='relu')(model)
    model = Dropout(0.5)(model)
    model = Dense(1, activation='linear')(model)
    
    model_ = Model(inputs=[first_input, second], outputs=model)

    model_.compile(optimizer='adam',
                  loss='mean_squared_error', metrics=['mae'])
    
    codes = []
    idx = []
    for picture_index in range(startIndex,startIndex + numberSamples):
        code = retrieved_image_df.iloc[picture_index].name
        codes.append(code)
        idx.append(picture_index)
    
    dset, tar, gen_codes, gen_indexes = data_set(codes, idx)
    dset = dset.apply(LabelEncoder().fit_transform)
#     dset = pd.get_dummies(dset)
    print(dset.shape)
       
    if batchSize>dset.shape[0]:
        batchSize = dset.shape[0]
        print('Batch size is automatically adjusted.')
    
    numOfBatches, lastBatch = divmod(dset.shape[0], batchSize-1)
    dsets = np.array_split(dset, numOfBatches)
    tars = np.array_split(tar, numOfBatches)
    gen_codes = np.array_split(gen_codes, numOfBatches)
    gen_indexes = np.array_split(gen_indexes, numOfBatches)
    
    for batch in range(numOfBatches):
        pic1 =[]
        print('Batch: '+str(batch+1)+'/'+str(numOfBatches))
        for code, picture_index in zip(gen_codes[batch], gen_indexes[batch]):
            im_index = retrieved_image_df.iloc[picture_index]['image1'+'_index']        
            if (math.isnan(float(im_index))==False):
                filename = str(code).lower()+'_'+'image1'+'_'+str(int(im_index))+'.jpg'
                path = '/Koton_Image_Files/'+filename
                try:
                    md, res = dbx.files_download(path)
                    temp_pic = cv2.cvtColor(cv2.resize(cv2.imdecode(np.frombuffer(res.content, dtype=np.uint8), -1), (299,299)), cv2.COLOR_BGR2RGB)
                    assert temp_pic.shape == (299, 299, 3)
                    pic1.append(temp_pic)
                except:
                    errorLog.append(path+' Not found')
                    batch += 1 #ignore defective batch
        pic1 = np.array(pic1)
        bottleneck_features = bn_model.predict(pic1)
        print(bottleneck_features.shape)
        print(dsets[batch].shape)
        try:
            model_.fit([bottleneck_features, dsets[batch]], tars[batch],
                  epochs=10)
        except:
            exc_type, exc_value, exc_traceback = sys.exc_info()
            lines = traceback.format_exception(exc_type, exc_value, exc_traceback)
            errorLog.append(''.join('!! ' + line for line in lines))  
      
    return model_, errorLog
                    


In [86]:
model, errorLog = getImages_n_train(900, 1724, 64)

NameError: name 'second_' is not defined

In [ ]:
errorLog

* increment querysinde aggregate / **stock azalmalarini dus**
* Correct image index. ok
* Check images ok
* price ekle

In [9]:
def getImages_n_train(numberSamples, startIndex, batchSize=64):
    errorLog = []
    bn_model = Xception(include_top=False, weights='imagenet', pooling='avg')  #, pooling='avg'
    
    first = Input(shape=(2048,))
#     first = Flatten()(first_input)
    
    second = Input(shape=(195,)) #change according to columns
#     second = Dense(10, activation='relu')(second_)
    
    model = concatenate([first, second], axis=1)
    model = BatchNormalization()(model)
    model = Dense(256, activation='relu')(model)
    model = Dropout(0.5)(model)
    model = Dense(256, activation='relu')(model)
    model = Dropout(0.5)(model)
    model = Dense(1, activation='linear')(model)
    
    model_ = Model(inputs=[first, second], outputs=model)

    model_.compile(optimizer='adam',
                  loss='mean_squared_error', metrics=['mae'])
    
    codes = []
    idx = []
    for picture_index in range(startIndex,startIndex + numberSamples):
        code = retrieved_image_df.iloc[picture_index].name
        codes.append(code)
        idx.append(picture_index)
    
    dset, tar, gen_codes, gen_indexes = data_set(codes, idx)
#     dset = dset.apply(LabelEncoder().fit_transform)
    dset = pd.get_dummies(dset)
    print(dset.shape)
       
    if batchSize>dset.shape[0]:
        batchSize = dset.shape[0]
        print('Batch size is automatically adjusted.')
    
    numOfBatches, lastBatch = divmod(dset.shape[0], batchSize-1)
    dsets = np.array_split(dset, numOfBatches)
    tars = np.array_split(tar, numOfBatches)
    gen_codes = np.array_split(gen_codes, numOfBatches)
    gen_indexes = np.array_split(gen_indexes, numOfBatches)
    
    for batch in range(numOfBatches):
        pic1 =[]
        print('Batch: '+str(batch+1)+'/'+str(numOfBatches))
        for code, picture_index in zip(gen_codes[batch], gen_indexes[batch]):
            im_index = retrieved_image_df.iloc[picture_index]['image1'+'_index']        
            if (math.isnan(float(im_index))==False):
                filename = str(code).lower()+'_'+'image1'+'_'+str(int(im_index))+'.jpg'
                path = '/Koton_Image_Files/'+filename
                try:
                    md, res = dbx.files_download(path)
                    temp_pic = cv2.cvtColor(cv2.resize(cv2.imdecode(np.frombuffer(res.content, dtype=np.uint8), -1), (299,299)), cv2.COLOR_BGR2RGB)
                    assert temp_pic.shape == (299, 299, 3)
                    pic1.append(temp_pic)
                except:
                    errorLog.append(path+' Not found')
                    batch += 1 #ignore defective batch
        pic1 = np.array(pic1)
        bottleneck_features = bn_model.predict(pic1)
        try:
            model_.fit([bottleneck_features, dsets[batch]], tars[batch],
                  epochs=10)
        except:
            exc_type, exc_value, exc_traceback = sys.exc_info()
            lines = traceback.format_exception(exc_type, exc_value, exc_traceback)
            errorLog.append(''.join('!! ' + line for line in lines))  
      
    return model_, errorLog
                    


In [10]:
model, errorLog = getImages_n_train(40000, 1724, 64)

(9071, 195)
Batch: 1/143
Epoch 1/10
64/64 [==============================] - 1s 17ms/step - loss: 49.0477 - mean_absolute_error: 2.6817
Epoch 2/10
64/64 [==============================] - 0s 438us/step - loss: 44.6657 - mean_absolute_error: 2.4617
Epoch 3/10
64/64 [==============================] - 0s 438us/step - loss: 41.6304 - mean_absolute_error: 2.4120
Epoch 4/10
64/64 [==============================] - 0s 469us/step - loss: 39.4827 - mean_absolute_error: 2.7916
Epoch 5/10
64/64 [==============================] - 0s 453us/step - loss: 43.5539 - mean_absolute_error: 2.6179
Epoch 6/10
64/64 [==============================] - 0s 469us/step - loss: 36.1577 - mean_absolute_error: 2.6189
Epoch 7/10
64/64 [==============================] - 0s 453us/step - loss: 35.4729 - mean_absolute_error: 2.5319
Epoch 8/10
64/64 [==============================] - 0s 453us/step - loss: 27.9162 - mean_absolute_error: 2.4840
Epoch 9/10
64/64 [==============================] - 0s 453us/step - loss: 29.509

64/64 [==============================] - 0s 469us/step - loss: 8.6305 - mean_absolute_error: 1.9468
Epoch 4/10
64/64 [==============================] - 0s 469us/step - loss: 8.6267 - mean_absolute_error: 1.9509
Epoch 5/10
64/64 [==============================] - 0s 453us/step - loss: 8.5597 - mean_absolute_error: 1.9261
Epoch 6/10
64/64 [==============================] - 0s 453us/step - loss: 8.5781 - mean_absolute_error: 1.9220
Epoch 7/10
64/64 [==============================] - 0s 469us/step - loss: 8.5807 - mean_absolute_error: 1.9297
Epoch 8/10
64/64 [==============================] - 0s 485us/step - loss: 8.6293 - mean_absolute_error: 1.9397
Epoch 9/10
64/64 [==============================] - 0s 453us/step - loss: 8.6439 - mean_absolute_error: 1.9530
Epoch 10/10
64/64 [==============================] - 0s 453us/step - loss: 8.4411 - mean_absolute_error: 1.9099
Batch: 11/143
Epoch 1/10
64/64 [==============================] - 0s 453us/step - loss: 10.5156 - mean_absolute_error: 1.9

64/64 [==============================] - 0s 453us/step - loss: 34.1302 - mean_absolute_error: 3.5200
Epoch 6/10
64/64 [==============================] - 0s 453us/step - loss: 34.1719 - mean_absolute_error: 3.5469
Epoch 7/10
64/64 [==============================] - 0s 469us/step - loss: 34.1139 - mean_absolute_error: 3.5240
Epoch 8/10
64/64 [==============================] - 0s 453us/step - loss: 34.1404 - mean_absolute_error: 3.5202
Epoch 9/10
64/64 [==============================] - 0s 438us/step - loss: 34.0157 - mean_absolute_error: 3.4860
Epoch 10/10
64/64 [==============================] - 0s 469us/step - loss: 34.1567 - mean_absolute_error: 3.5328
Batch: 18/143
Batch: 19/143
Epoch 1/10
64/64 [==============================] - 0s 453us/step - loss: 116.2135 - mean_absolute_error: 6.7958
Epoch 2/10
64/64 [==============================] - 0s 485us/step - loss: 115.0707 - mean_absolute_error: 6.6152
Epoch 3/10
64/64 [==============================] - 0s 469us/step - loss: 117.9855 -

64/64 [==============================] - 0s 453us/step - loss: 6.9242 - mean_absolute_error: 1.9051
Epoch 8/10
64/64 [==============================] - 0s 469us/step - loss: 6.8275 - mean_absolute_error: 1.8422
Epoch 9/10
64/64 [==============================] - 0s 453us/step - loss: 5.7514 - mean_absolute_error: 1.7589
Epoch 10/10
64/64 [==============================] - 0s 453us/step - loss: 5.3062 - mean_absolute_error: 1.7147
Batch: 26/143
Epoch 1/10
64/64 [==============================] - 0s 453us/step - loss: 17.8641 - mean_absolute_error: 2.5914
Epoch 2/10
64/64 [==============================] - 0s 516us/step - loss: 19.4876 - mean_absolute_error: 2.8819
Epoch 3/10
64/64 [==============================] - 0s 516us/step - loss: 19.2232 - mean_absolute_error: 2.8904
Epoch 4/10
64/64 [==============================] - 0s 500us/step - loss: 18.3882 - mean_absolute_error: 2.7927
Epoch 5/10
64/64 [==============================] - 0s 516us/step - loss: 19.5747 - mean_absolute_error:

64/64 [==============================] - 0s 438us/step - loss: 6.3428 - mean_absolute_error: 2.1305
Epoch 10/10
64/64 [==============================] - 0s 438us/step - loss: 7.1088 - mean_absolute_error: 2.1237
Batch: 35/143
Epoch 1/10
64/64 [==============================] - 0s 469us/step - loss: 13.8239 - mean_absolute_error: 2.7748
Epoch 2/10
64/64 [==============================] - 0s 438us/step - loss: 11.9959 - mean_absolute_error: 2.4594
Epoch 3/10
64/64 [==============================] - 0s 438us/step - loss: 13.0294 - mean_absolute_error: 2.6019
Epoch 4/10
64/64 [==============================] - 0s 422us/step - loss: 13.1859 - mean_absolute_error: 2.5551
Epoch 5/10
64/64 [==============================] - 0s 422us/step - loss: 10.6188 - mean_absolute_error: 2.3727
Epoch 6/10
64/64 [==============================] - 0s 438us/step - loss: 9.6543 - mean_absolute_error: 2.2452
Epoch 7/10
64/64 [==============================] - 0s 407us/step - loss: 8.3520 - mean_absolute_error:

...

63/63 [==============================] - 0s 461us/step - loss: 182.7851 - mean_absolute_error: 7.4547
Epoch 2/10
63/63 [==============================] - 0s 461us/step - loss: 191.3469 - mean_absolute_error: 7.9032
Epoch 3/10
63/63 [==============================] - 0s 492us/step - loss: 187.6904 - mean_absolute_error: 7.6734
Epoch 4/10
63/63 [==============================] - 0s 524us/step - loss: 187.3165 - mean_absolute_error: 7.5184
Epoch 5/10
63/63 [==============================] - 0s 508us/step - loss: 173.9727 - mean_absolute_error: 7.2483
Epoch 6/10
63/63 [==============================] - 0s 508us/step - loss: 178.5903 - mean_absolute_error: 7.4672
Epoch 7/10
63/63 [==============================] - 0s 524us/step - loss: 170.1486 - mean_absolute_error: 7.0660
Epoch 8/10
63/63 [==============================] - 0s 508us/step - loss: 183.5583 - mean_absolute_error: 7.6948
Epoch 9/10
63/63 [==============================] - 0s 540us/step - loss: 179.4231 - mean_absolute_error: 7

63/63 [==============================] - 0s 429us/step - loss: 291.2481 - mean_absolute_error: 10.3964
Epoch 3/10
63/63 [==============================] - 0s 445us/step - loss: 265.9276 - mean_absolute_error: 9.6841
Epoch 4/10
63/63 [==============================] - 0s 477us/step - loss: 284.1412 - mean_absolute_error: 10.5835
Epoch 5/10
63/63 [==============================] - 0s 461us/step - loss: 251.3786 - mean_absolute_error: 9.5834
Epoch 6/10
63/63 [==============================] - 0s 461us/step - loss: 259.3267 - mean_absolute_error: 9.8063
Epoch 7/10
63/63 [==============================] - 0s 477us/step - loss: 304.4449 - mean_absolute_error: 10.6321
Epoch 8/10
63/63 [==============================] - 0s 477us/step - loss: 279.9803 - mean_absolute_error: 9.7512
Epoch 9/10
63/63 [==============================] - 0s 461us/step - loss: 237.7832 - mean_absolute_error: 9.5131
Epoch 10/10
63/63 [==============================] - 0s 461us/step - loss: 246.6838 - mean_absolute_erro

63/63 [==============================] - 0s 461us/step - loss: 351.6756 - mean_absolute_error: 13.5201
Epoch 4/10
63/63 [==============================] - 0s 477us/step - loss: 328.7193 - mean_absolute_error: 12.6510
Epoch 5/10
63/63 [==============================] - 0s 445us/step - loss: 274.4048 - mean_absolute_error: 11.8917
Epoch 6/10
63/63 [==============================] - 0s 492us/step - loss: 291.6689 - mean_absolute_error: 12.5378
Epoch 7/10
63/63 [==============================] - 0s 477us/step - loss: 309.0837 - mean_absolute_error: 12.0815
Epoch 8/10
63/63 [==============================] - 0s 445us/step - loss: 329.6692 - mean_absolute_error: 13.5170
Epoch 9/10
63/63 [==============================] - 0s 461us/step - loss: 291.6220 - mean_absolute_error: 12.0556
Epoch 10/10
63/63 [==============================] - 0s 492us/step - loss: 245.5287 - mean_absolute_error: 12.1051
Batch: 124/143
Epoch 1/10
63/63 [==============================] - 0s 461us/step - loss: 1194.7385

63/63 [==============================] - 0s 477us/step - loss: 112.0077 - mean_absolute_error: 6.9537
Epoch 5/10
63/63 [==============================] - 0s 492us/step - loss: 127.5558 - mean_absolute_error: 7.2983
Epoch 6/10
63/63 [==============================] - 0s 508us/step - loss: 116.8347 - mean_absolute_error: 7.0185
Epoch 7/10
63/63 [==============================] - 0s 492us/step - loss: 120.6060 - mean_absolute_error: 7.1635
Epoch 8/10
63/63 [==============================] - 0s 540us/step - loss: 115.0104 - mean_absolute_error: 6.9487
Epoch 9/10
63/63 [==============================] - 0s 508us/step - loss: 109.1725 - mean_absolute_error: 7.0639
Epoch 10/10
63/63 [==============================] - 0s 492us/step - loss: 123.3274 - mean_absolute_error: 7.5886
Batch: 133/143
Epoch 1/10
63/63 [==============================] - 0s 429us/step - loss: 198.4845 - mean_absolute_error: 8.4131
Epoch 2/10
63/63 [==============================] - 0s 461us/step - loss: 136.9054 - mean_a

63/63 [==============================] - 0s 508us/step - loss: 93.4472 - mean_absolute_error: 6.3239
Epoch 6/10
63/63 [==============================] - 0s 492us/step - loss: 105.5724 - mean_absolute_error: 6.7651
Epoch 7/10
63/63 [==============================] - 0s 508us/step - loss: 98.9138 - mean_absolute_error: 6.3838
Epoch 8/10
63/63 [==============================] - 0s 477us/step - loss: 95.3577 - mean_absolute_error: 6.1315
Epoch 9/10
63/63 [==============================] - 0s 508us/step - loss: 90.3216 - mean_absolute_error: 5.5943
Epoch 10/10
63/63 [==============================] - 0s 492us/step - loss: 77.8528 - mean_absolute_error: 5.0369


In [15]:
def getImages_n_train(numberSamples, startIndex, batchSize=64):
    errorLog = []
    bn_model = Xception(include_top=False, weights='imagenet', pooling='max')  #, pooling='avg'
    
    first = Input(shape=(2048,))
#     first = Flatten()(first_input)
    
    second = Input(shape=(195,)) #change according to columns
#     second = Dense(10, activation='relu')(second_)
    
    model = concatenate([first, second], axis=1)
    model = BatchNormalization()(model)
    model = Dense(256, activation='relu', kernel_initializer='he_uniform')(model)
    model = Dropout(0.5)(model)
    model = Dense(256, activation='relu', kernel_initializer='he_uniform')(model)
    model = Dropout(0.5)(model)
    model = Dense(1, activation='linear')(model)
    
    model_ = Model(inputs=[first, second], outputs=model)

    model_.compile(optimizer='adam',
                  loss='mean_squared_error', metrics=['mae'])
    
    codes = []
    idx = []
    for picture_index in range(startIndex,startIndex + numberSamples):
        code = retrieved_image_df.iloc[picture_index].name
        codes.append(code)
        idx.append(picture_index)
    
    dset, tar, gen_codes, gen_indexes = data_set(codes, idx)
#     dset = dset.apply(LabelEncoder().fit_transform)
    dset = pd.get_dummies(dset)
    print(dset.shape)
       
    if batchSize>dset.shape[0]:
        batchSize = dset.shape[0]
        print('Batch size is automatically adjusted.')
    
    numOfBatches, lastBatch = divmod(dset.shape[0], batchSize)
    dsets = np.array_split(dset, numOfBatches)
    tars = np.array_split(tar, numOfBatches)
    gen_codes = np.array_split(gen_codes, numOfBatches)
    gen_indexes = np.array_split(gen_indexes, numOfBatches)
    
    for batch in range(numOfBatches):
        pic1 =[]
        print('Batch: '+str(batch+1)+'/'+str(numOfBatches))
        for code, picture_index in zip(gen_codes[batch], gen_indexes[batch]):
            im_index = retrieved_image_df.iloc[picture_index]['image1'+'_index']        
            if (math.isnan(float(im_index))==False):
                filename = str(code).lower()+'_'+'image1'+'_'+str(int(im_index))+'.jpg'
                path = '/Koton_Image_Files/'+filename
                try:
                    md, res = dbx.files_download(path)
                    temp_pic = cv2.cvtColor(cv2.resize(cv2.imdecode(np.frombuffer(res.content, dtype=np.uint8), -1), (299,299)), cv2.COLOR_BGR2RGB)
                    assert temp_pic.shape == (299, 299, 3)
                    pic1.append(temp_pic)
                except:
                    errorLog.append(path+' Not found')
                    batch += 1 #ignore defective batch
        pic1 = np.array(pic1)
        bottleneck_features = bn_model.predict(pic1)
        try:
            model_.fit([bottleneck_features, dsets[batch]], tars[batch],
                  epochs=10)
        except:
            exc_type, exc_value, exc_traceback = sys.exc_info()
            lines = traceback.format_exception(exc_type, exc_value, exc_traceback)
            errorLog.append(''.join('!! ' + line for line in lines))  
      
    return model_, errorLog
                    


In [16]:
model, errorLog = getImages_n_train(40000, 1724, 64)

(9071, 195)
Batch: 1/141
Epoch 1/10
65/65 [==============================] - 3s 39ms/step - loss: 57.8846 - mean_absolute_error: 3.4873
Epoch 2/10
65/65 [==============================] - 0s 647us/step - loss: 48.5347 - mean_absolute_error: 3.4512
Epoch 3/10
65/65 [==============================] - 0s 662us/step - loss: 42.4146 - mean_absolute_error: 3.2292
Epoch 4/10
65/65 [==============================] - 0s 662us/step - loss: 38.6215 - mean_absolute_error: 3.5699
Epoch 5/10
65/65 [==============================] - 0s 647us/step - loss: 36.1652 - mean_absolute_error: 3.2352
Epoch 6/10
65/65 [==============================] - 0s 647us/step - loss: 22.8099 - mean_absolute_error: 2.9691
Epoch 7/10
65/65 [==============================] - 0s 631us/step - loss: 26.9843 - mean_absolute_error: 3.0699
Epoch 8/10
65/65 [==============================] - 0s 647us/step - loss: 25.1731 - mean_absolute_error: 3.1975
Epoch 9/10
65/65 [==============================] - 0s 631us/step - loss: 15.308

65/65 [==============================] - 0s 647us/step - loss: 6.6160 - mean_absolute_error: 1.7058
Epoch 3/10
65/65 [==============================] - 0s 616us/step - loss: 6.4409 - mean_absolute_error: 1.7130
Epoch 4/10
65/65 [==============================] - 0s 616us/step - loss: 6.5743 - mean_absolute_error: 1.6089
Epoch 5/10
65/65 [==============================] - 0s 647us/step - loss: 4.7756 - mean_absolute_error: 1.6172
Epoch 6/10
65/65 [==============================] - 0s 616us/step - loss: 4.3595 - mean_absolute_error: 1.4193
Epoch 7/10
65/65 [==============================] - 0s 631us/step - loss: 6.2696 - mean_absolute_error: 1.5895
Epoch 8/10
65/65 [==============================] - 0s 647us/step - loss: 3.8105 - mean_absolute_error: 1.3678
Epoch 9/10
65/65 [==============================] - 0s 631us/step - loss: 5.7727 - mean_absolute_error: 1.6442
Epoch 10/10
65/65 [==============================] - 0s 631us/step - loss: 3.3575 - mean_absolute_error: 1.3485
Batch: 11/1

65/65 [==============================] - 0s 647us/step - loss: 19.9355 - mean_absolute_error: 3.1906
Epoch 5/10
65/65 [==============================] - 0s 647us/step - loss: 18.4338 - mean_absolute_error: 2.6400
Epoch 6/10
65/65 [==============================] - 0s 647us/step - loss: 14.4684 - mean_absolute_error: 2.4443
Epoch 7/10
65/65 [==============================] - 0s 631us/step - loss: 19.4359 - mean_absolute_error: 2.8264
Epoch 8/10
65/65 [==============================] - 0s 631us/step - loss: 10.8324 - mean_absolute_error: 2.5062
Epoch 9/10
65/65 [==============================] - 0s 647us/step - loss: 12.6282 - mean_absolute_error: 2.5040
Epoch 10/10
65/65 [==============================] - 0s 662us/step - loss: 21.3937 - mean_absolute_error: 2.6489
Batch: 18/141
Batch: 19/141
Epoch 1/10
65/65 [==============================] - 0s 631us/step - loss: 88.8955 - mean_absolute_error: 5.5049
Epoch 2/10
65/65 [==============================] - 0s 708us/step - loss: 85.6586 - me

65/65 [==============================] - 0s 647us/step - loss: 7.6426 - mean_absolute_error: 1.9322
Epoch 7/10
65/65 [==============================] - 0s 647us/step - loss: 8.3011 - mean_absolute_error: 1.9569
Epoch 8/10
65/65 [==============================] - 0s 662us/step - loss: 8.3381 - mean_absolute_error: 2.0721
Epoch 9/10
65/65 [==============================] - 0s 662us/step - loss: 7.2732 - mean_absolute_error: 1.8755
Epoch 10/10
65/65 [==============================] - 0s 647us/step - loss: 5.1006 - mean_absolute_error: 1.5040
Batch: 26/141
Epoch 1/10
65/65 [==============================] - 0s 662us/step - loss: 11.2335 - mean_absolute_error: 2.2223
Epoch 2/10
65/65 [==============================] - 0s 677us/step - loss: 14.1481 - mean_absolute_error: 2.6295
Epoch 3/10
65/65 [==============================] - 0s 631us/step - loss: 12.3490 - mean_absolute_error: 2.3748
Epoch 4/10
65/65 [==============================] - 0s 677us/step - loss: 10.2402 - mean_absolute_error: 

65/65 [==============================] - 0s 631us/step - loss: 3.6593 - mean_absolute_error: 1.4203
Epoch 9/10
65/65 [==============================] - 0s 677us/step - loss: 3.6260 - mean_absolute_error: 1.3871
Epoch 10/10
65/65 [==============================] - 0s 647us/step - loss: 3.8041 - mean_absolute_error: 1.3357
Batch: 34/141
Epoch 1/10
65/65 [==============================] - 0s 647us/step - loss: 5.3467 - mean_absolute_error: 1.6317
Epoch 2/10
65/65 [==============================] - 0s 631us/step - loss: 6.9609 - mean_absolute_error: 1.8272
Epoch 3/10
65/65 [==============================] - 0s 631us/step - loss: 3.8137 - mean_absolute_error: 1.4640
Epoch 4/10
65/65 [==============================] - 0s 647us/step - loss: 4.4695 - mean_absolute_error: 1.6204
Epoch 5/10
65/65 [==============================] - 0s 631us/step - loss: 3.6568 - mean_absolute_error: 1.4166
Epoch 6/10
65/65 [==============================] - 0s 631us/step - loss: 3.7357 - mean_absolute_error: 1.41

...

64/64 [==============================] - 0s 485us/step - loss: 134.0201 - mean_absolute_error: 7.2908
Batch: 98/141
Epoch 1/10
64/64 [==============================] - 0s 485us/step - loss: 87.2109 - mean_absolute_error: 6.3957
Epoch 2/10
64/64 [==============================] - 0s 485us/step - loss: 96.0817 - mean_absolute_error: 6.9256
Epoch 3/10
64/64 [==============================] - 0s 469us/step - loss: 86.5526 - mean_absolute_error: 6.6794
Epoch 4/10
64/64 [==============================] - 0s 469us/step - loss: 81.1159 - mean_absolute_error: 6.3771
Epoch 5/10
64/64 [==============================] - 0s 469us/step - loss: 76.5644 - mean_absolute_error: 6.2649
Epoch 6/10
64/64 [==============================] - 0s 469us/step - loss: 74.3369 - mean_absolute_error: 6.3624
Epoch 7/10
64/64 [==============================] - 0s 469us/step - loss: 67.8338 - mean_absolute_error: 5.8170
Epoch 8/10
64/64 [==============================] - 0s 469us/step - loss: 64.4360 - mean_absolute_er

64/64 [==============================] - 0s 469us/step - loss: 524.8848 - mean_absolute_error: 11.8905
Epoch 2/10
64/64 [==============================] - 0s 485us/step - loss: 484.3870 - mean_absolute_error: 11.6730
Epoch 3/10
64/64 [==============================] - 0s 485us/step - loss: 481.3628 - mean_absolute_error: 11.0899
Epoch 4/10
64/64 [==============================] - 0s 485us/step - loss: 473.0943 - mean_absolute_error: 11.0797
Epoch 5/10
64/64 [==============================] - 0s 469us/step - loss: 480.9009 - mean_absolute_error: 11.3722
Epoch 6/10
64/64 [==============================] - 0s 469us/step - loss: 459.8307 - mean_absolute_error: 11.1009
Epoch 7/10
64/64 [==============================] - 0s 500us/step - loss: 450.4752 - mean_absolute_error: 11.5795
Epoch 8/10
64/64 [==============================] - 0s 469us/step - loss: 443.8660 - mean_absolute_error: 10.8214
Epoch 9/10
64/64 [==============================] - 0s 469us/step - loss: 436.7808 - mean_absolute_

64/64 [==============================] - 0s 500us/step - loss: 81.9292 - mean_absolute_error: 7.0377
Epoch 3/10
64/64 [==============================] - 0s 469us/step - loss: 82.6428 - mean_absolute_error: 7.0013
Epoch 4/10
64/64 [==============================] - 0s 469us/step - loss: 89.7818 - mean_absolute_error: 6.9100
Epoch 5/10
64/64 [==============================] - 0s 469us/step - loss: 76.2274 - mean_absolute_error: 6.5109
Epoch 6/10
64/64 [==============================] - 0s 469us/step - loss: 76.6363 - mean_absolute_error: 6.8448
Epoch 7/10
64/64 [==============================] - 0s 469us/step - loss: 92.3870 - mean_absolute_error: 7.2003
Epoch 8/10
64/64 [==============================] - 0s 469us/step - loss: 84.8182 - mean_absolute_error: 6.8663
Epoch 9/10
64/64 [==============================] - 0s 469us/step - loss: 74.4761 - mean_absolute_error: 6.6025
Epoch 10/10
64/64 [==============================] - 0s 469us/step - loss: 75.6874 - mean_absolute_error: 6.4750
Ba

64/64 [==============================] - 0s 500us/step - loss: 177.1897 - mean_absolute_error: 10.6488
Epoch 4/10
64/64 [==============================] - 0s 500us/step - loss: 107.8942 - mean_absolute_error: 8.5172
Epoch 5/10
64/64 [==============================] - 0s 500us/step - loss: 125.0980 - mean_absolute_error: 8.7197
Epoch 6/10
64/64 [==============================] - 0s 485us/step - loss: 102.9644 - mean_absolute_error: 7.6984
Epoch 7/10
64/64 [==============================] - 0s 500us/step - loss: 112.8364 - mean_absolute_error: 8.5575
Epoch 8/10
64/64 [==============================] - 0s 485us/step - loss: 108.9910 - mean_absolute_error: 7.7848
Epoch 9/10
64/64 [==============================] - 0s 485us/step - loss: 87.2673 - mean_absolute_error: 7.1847
Epoch 10/10
64/64 [==============================] - 0s 500us/step - loss: 91.1134 - mean_absolute_error: 7.2158
Batch: 125/141
Epoch 1/10
64/64 [==============================] - 0s 485us/step - loss: 175.6908 - mean_ab

64/64 [==============================] - 0s 485us/step - loss: 130.2042 - mean_absolute_error: 8.2439
Epoch 5/10
64/64 [==============================] - 0s 500us/step - loss: 136.6087 - mean_absolute_error: 8.3219
Epoch 6/10
64/64 [==============================] - 0s 516us/step - loss: 124.7035 - mean_absolute_error: 8.4593
Epoch 7/10
64/64 [==============================] - 0s 485us/step - loss: 124.8452 - mean_absolute_error: 8.3408
Epoch 8/10
64/64 [==============================] - 0s 485us/step - loss: 126.0567 - mean_absolute_error: 8.2302
Epoch 9/10
64/64 [==============================] - 0s 500us/step - loss: 106.2285 - mean_absolute_error: 7.6212
Epoch 10/10
64/64 [==============================] - 0s 485us/step - loss: 110.6400 - mean_absolute_error: 7.1870
Batch: 135/141
Epoch 1/10
64/64 [==============================] - 0s 500us/step - loss: 241.0105 - mean_absolute_error: 8.7717
Epoch 2/10
64/64 [==============================] - 0s 500us/step - loss: 274.5067 - mean_a